In [36]:
from sentence_transformers import CrossEncoder
from langchain_core.documents import Document

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')


In [21]:
import numpy as np

In [19]:
query = "I wrote my programm in Python"

docs = ["Hello WOrld!", "C programming and POSIX", "Good Morning!"]



pairs = [[query, doc] for doc in docs]
scores = cross_encoder.predict(pairs)
print("Scores:")
scores

Scores:


array([-10.616302 ,  -8.4896145, -10.841426 ], dtype=float32)

In [33]:
np.argsort(scores)[::-1]

array([1, 0, 2])

In [42]:
class CrossEncoderReRanker():
    def __init__(self, cross_encoder):
        self.cross_encoder = cross_encoder
        
    def _compute_scores(self, query, documents):
        
        pairs = [[query, doc.page_content] for doc in documents]
        scores = self.cross_encoder.predict(pairs)
        
        return scores
    
    def get_most_relevant_chunks(self, query, documents, n):
        
        scores = self._compute_scores(query, documents)
        sorted_indices = np.argsort(scores)[::-1]
        
        relevant_docs = [documents[i] for i in sorted_indices[:n]]
        
        return relevant_docs
        
        
    
    

In [43]:
reranker = CrossEncoderReRanker(cross_encoder)


In [44]:
doc1 = Document(page_content="I have been doing Python programming since 2021.I use Django for Web Development",
                metadata={"page" : 1})

doc2 = Document(page_content="My name is Stas. I live in Bangkok",
                metadata={"page" : 1})




In [47]:
reranker.get_most_relevant_chunks("Do you like Thailand?", [doc1, doc2], n=2)

[Document(page_content='My name is Stas. I live in Bangkok', metadata={'page': 1}),
 Document(page_content='I have been doing Python programming since 2021.I use Django for Web Development', metadata={'page': 1})]